In [7]:
import pandas as pd
import numpy as np
import glob
import subprocess

import biopandas as bio

In [2]:
def get_fasta(pdbmame):
    dictionary = {'ALA':'A','ARG':'R','ASN':'N','ASP':'D', \
                  'CYS':'C','GLU':'E','GLN':'Q','GLY':'G', \
                  'HIS':'H','ILE':'I','LEU':'L','LYS':'K', \
                  'MET':'M','PHE':'F','PRO':'P','SER':'S', \
                  'THR':'T','TRP':'W','TYR':'Y','VAL':'V',
                  'HIE':'H','HID':'H','HIP':'H'}

    pdbname = pdbmame

    lines = open(pdbname,'r').readlines()

    oldresnum = '   '
    count = 1;

    list_fasta = []
    for line in lines:
            line_edit = line
            if line[0:3] == 'TER':
               count = 1
               continue

            if line_edit[0:4] == 'ATOM':
                    if 'N  ' == line_edit[13:16]:
                        count = count + 1
                        resn = line[17:20]
                        try:
                            list_fasta.append(dictionary[resn])
                            if count==61:
                             count = 1
                        except:
                          pass
    return list_fasta

In [5]:
lista_df = []
for i in glob.glob('/pdbmani/Serch/Experimentos/cluster_*'):
    for j in glob.glob(i+'/*'):
        nombre1 = (j.split('/')[-1][:6]+'.pdb')
        nombre2 = (j.split('/')[-1][7:]+'.pdb')

        print(j+'/'+nombre1)
        print(j+'/'+nombre2)
        
        try:
            lista_1 = get_fasta(j+'/'+nombre1)
            lista_2 = get_fasta(j+'/'+nombre2)
        except FileNotFoundError:
            continue

        lista_identity = []
        for val1, val2 in enumerate(zip(lista_1,lista_2)):
            if val2[0] == val2[1]:
                lista_identity.append(1)
            else:
                lista_identity.append(0)
        
        val_identity = np.round(np.mean(lista_identity),4)*100
        val_id_protein = j.split('/')[-1]
        
        print(val_identity)
        print(val_id_protein)
        
        lista_df.append([val_id_protein,val_identity])

In [6]:
df = pd.DataFrame(lista_df,columns=['proteinaA_proteinaB','identity_score'])
df.head()

,proteinaA_proteinaB,identity_score


In [50]:
df1 = pd.read_csv('clickvsmani_all_exp.csv')

In [61]:
df_new = df1.merge(df,on='proteinaA_proteinaB').drop('Unnamed: 0',1)

In [63]:
df_new.to_csv('clickvsmani_all_exp_with_identity.csv',index=False)